In [1]:
import requests
import pandas as pd
import streamlit as st
import requests
import plotly.express as px
import warnings

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    warnings.warn("This is hidden")
    
url = "https://api.data.gov.in/resource/9ef84268-d588-465a-a308-a864a43d0070" # mandi api

params = {
    "api-key": "579b464db66ec23bdd000001683b398a0bdd40066aefc6ace98749c7",
    "format": "json",
    "limit": 10000
}

response = requests.get(url, params)

# Check status
print(response.status_code)

# Convert JSON → Python object
data = response.json()

# first 5 records
new_df = pd.DataFrame(data["records"]).reset_index()
new_df.head(2)

200


,index,state,district,market,commodity,variety,grade,arrival_date,min_price,max_price,modal_price
0,0,Punjab,Mohali,Kharar APMC,Raddish,Raddish,Local,26/12/2025,700.0,1000.0,800.0
1,1,Punjab,Mohali,Kharar APMC,Carrot,Carrot,Local,26/12/2025,800.0,1200.0,1000.0


In [2]:
new_df.tail(2)

,index,state,district,market,commodity,variety,grade,arrival_date,min_price,max_price,modal_price
7181,7181,Himachal Pradesh,Mandi,SMY Jogindernagar,Raddish,Other,Grade B,26/12/2025,1200.0,1500.0,1350.0
7182,7182,Rajasthan,Dudu,Dudu APMC,Green Gram(Moong)(Whole),Local,Local,26/12/2025,4030.0,4030.0,4030.0


In [3]:
import os

FILE_PATH = "market_data_master.csv"

COLUMNS = [
    "state", "district", "market", "commodity", "variety", "grade",
    "arrival_date", "min_price", "max_price", "modal_price"
]

# Create empty CSV if it doesn't exist
if not os.path.exists(FILE_PATH):
    empty_df = pd.DataFrame(columns=COLUMNS)
    empty_df.to_csv(FILE_PATH, index=False)

existing_df = pd.read_csv("market_data_master.csv")

df = pd.concat([existing_df, new_df], ignore_index=True)
updated_df = df.copy()
updated_df = updated_df.to_csv(FILE_PATH, index=False)

C:\Users\SudarshanMhatre\AppData\Local\Temp\ipykernel_11240\3794046688.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([existing_df, new_df], ignore_index=True)


In [8]:
state = df.groupby('state').sum('modal_price')

In [9]:
state

,min_price,max_price,modal_price,index
state,,,,
Andhra Pradesh,1.074920e+05,1.435920e+05,1.296920e+05,120941.0
Assam,3.432268e+05,4.184224e+05,3.787204e+05,284971.0
Bihar,3.590000e+04,4.360000e+04,3.950000e+04,28459.0
Gujarat,1.160485e+06,1.583775e+06,1.399159e+06,1136836.0
Haryana,7.629710e+05,9.735060e+05,8.668200e+05,1136767.0
Himachal Pradesh,1.552400e+06,1.944500e+06,1.751530e+06,1498299.0
Jammu and Kashmir,1.850500e+05,2.013500e+05,1.932000e+05,169482.0
Karnataka,8.300000e+03,9.730000e+03,9.283000e+03,5157.0
Kerala,3.240370e+06,3.625290e+06,3.427230e+06,1671483.0


In [10]:
# ----------------- 2. Sidebar Filters -----------------
st.sidebar.header("Filters")

states = st.sidebar.multiselect("Select States", df['state'].unique(), default=df['state'].unique())
districts = st.sidebar.multiselect("Select Districts", df['district'].unique(), default=df['district'].unique())
commodities = st.sidebar.multiselect("Select Commodities", df['commodity'].unique(), default=df['commodity'].unique())
varieties = st.sidebar.multiselect("Select Varieties", df['variety'].unique(), default=df['variety'].unique())
grades = st.sidebar.multiselect("Select Grades", df['grade'].unique(), default=df['grade'].unique())

filtered_df = df[
    (df['state'].isin(states)) &
    (df['district'].isin(districts)) &
    (df['commodity'].isin(commodities)) &
    (df['variety'].isin(varieties)) &
    (df['grade'].isin(grades))
]

# ----------------- 3. Dashboard Title -----------------
st.title("📊 Indian Market Prices Dashboard")
st.markdown("Interactive dashboard with all metrics, aggregations, and visualizations.")

# ----------------- 4. Display Filtered Table -----------------
st.subheader("Filtered Market Data")
st.dataframe(filtered_df)

# ----------------- 5. Price Metrics -----------------
st.subheader("Price Metrics / Calculations")

metrics_df = pd.DataFrame({
    "Metric": ["Min Price", "Max Price", "Modal Price Mean", "Modal Price Median",
               "Modal Price Std Dev", "Number of Records", "Price Range (Max-Min)"],
    "Value": [
        filtered_df['min_price'].min(),
        filtered_df['max_price'].max(),
        round(filtered_df['modal_price'].mean(),2),
        round(filtered_df['modal_price'].median(),2),
        round(filtered_df['modal_price'].std(),2),
        filtered_df.shape[0],
        filtered_df['max_price'].max() - filtered_df['min_price'].min()
    ]
})

st.table(metrics_df)

# ----------------- 6. Aggregations -----------------
st.subheader("Aggregated Data by State & Commodity")
agg_df = filtered_df.groupby(['state','commodity']).agg(
    Min_Price=('min_price','min'),
    Max_Price=('max_price','max'),
    Avg_Modal=('modal_price','mean'),
    Count=('modal_price','count')
).reset_index()
st.dataframe(agg_df)

# ----------------- 7. Visualizations -----------------
st.subheader("Visualizations")

# 7a: Commodity-wise Modal Prices Bar Chart
fig1 = px.bar(filtered_df, x="commodity", y="modal_price", color="state", text="modal_price",
              labels={"modal_price":"Modal Price", "commodity":"Commodity", "state":"State"},
              title="Commodity-wise Modal Prices by State")
fig1.update_layout(xaxis_title="commodity", yaxis_title="modal price")
st.plotly_chart(fig1)

# 7b: State-wise Average Modal Price Line Chart
state_avg = filtered_df.groupby("state")["modal_price"].mean().reset_index()
fig2 = px.line(state_avg, x="state", y="modal_price", markers=True,
               labels={"modal_price":"Average Modal Price", "state":"State"},
               title="State-wise Average Modal Price")
st.plotly_chart(fig2)

# 7c: Box Plot for Price Distribution
st.subheader("Price Distribution by Commodity")
fig3 = px.box(filtered_df, x="commodity", y="modal_price", color="state",
              labels={"modal_price":"Modal Price", "commodity":"Commodity", "state":"State"},
              title="Price Distribution by Commodity")
st.plotly_chart(fig3)

# 7d: Scatter Plot Min vs Max Prices
st.subheader("Min vs Max Prices Scatter Plot")
fig4 = px.scatter(filtered_df, x="min_price", y="max_price", color="commodity", size="modal_price",
                  hover_data=["state","district","market"], labels={"min_price":"Min Price","max_price":"Max Price"},
                  title="Scatter of Min vs Max Prices by Commodity")
st.plotly_chart(fig4)

# 7e: Pie Chart of Commodities Proportion
st.subheader("Commodity Share in Selected Data")
commodity_count = filtered_df['commodity'].value_counts().reset_index()
commodity_count.columns = ['commodity','count']
fig5 = px.pie(commodity_count, names='commodity', values='count', title='Commodity Proportion')
st.plotly_chart(fig5)

2025-12-26 12:56:43.043 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-26 12:56:43.044 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-26 12:56:43.045 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-26 12:56:43.047 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-26 12:56:43.047 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-26 12:56:43.049 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-26 12:56:43.050 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-26 12:56:43.050 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

DeltaGenerator()

In [11]:
!jupyter nbconvert --to script Daily_Price_Mandi.ipynb

[NbConvertApp] Converting notebook Daily_Price_Mandi.ipynb to script
[NbConvertApp] Writing 5549 bytes to Daily_Price_Mandi.py
